In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# to display full text
pd.set_option('display.max_colwidth', -1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Features_for_gcn_after_removing_duplicates_german.csv')

In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
df.shape

In [ ]:
df.drop(207, axis = 0, inplace = True)

In [ ]:
df.shape

In [ ]:
X = df.iloc[:, :-1].values #Take all the columns except last one
y = df.iloc[:, -1].values #Take the last column as the result

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
n1 = df['target_variable']
classes = 2
y1 = np.zeros((len(df),classes))
#labels
n = list(n1)
for i in range(len(df)):
    if (n[i] == 1):
        y1[i,1] = 1
    elif (n[i] == 0):
        y1[i,0] = 1

In [ ]:
y1.shape

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from scipy import interp
from sklearn.metrics import roc_auc_score
from tensorflow.keras import regularizers
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn import metrics

In [ ]:
# define 10-fold cross validation test harness
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
tp = []
tn = []
fp = []
fn = []

metrics_acc = []
metrics_precision = []
metrics_recall = []
metrics_f1 = []
pmacro = []
pmicro = []
pbinary = []
rmacro = []
rmicro = []
rbi = []
a = []
f = []

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
fig, ax = plt.subplots()
for i, (train, test) in enumerate(kfold.split(X, y)):
    print("i : ", i)
  #standardize
    sc_X = StandardScaler() 
    X[train] = sc_X.fit_transform(X[train]) 
    X[test] = sc_X.transform(X[test])

    X_train = np.reshape(X[train], (X[train].shape[0], 1, X[train].shape[1]))
    X_test = np.reshape(X[test], (X[test].shape[0], 1, X[test].shape[1]))
    # create model
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(32, return_sequences = True))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2,activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y1[train], epochs=300)

    classes = 2
    n_classes = 2
    #scores = model.evaluate(X_test, y[test], verbose=0)
    y_pred = model.predict(X_test)
    predict_class = np.argmax(y_pred, axis=1)

    y2 = np.zeros((len(test),classes))
    for i in range(len(test)):
        if (predict_class[i] == 1):
            y2[i,1] = 1
        elif (predict_class[i] == 0):
            y2[i,0] = 1

    fpr, tpr, t = roc_curve(y[test], predict_class)
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    
#auc roc curve
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y1[test, i], y2[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y1[test].ravel(), y2.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes
    lw = 2
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
            label='micro-average ROC curve (area = {0:0.2f})'
                  ''.format(roc_auc["micro"]),
            color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
            label='macro-average ROC curve (area = {0:0.2f})'
                  ''.format(roc_auc["macro"]),
            color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                label='ROC curve of class {0} (area = {1:0.2f})'
                ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.show()
    
    print(predict_class)
    conf_mat = confusion_matrix(y[test], predict_class)
    
    print(conf_mat)

    p = precision_score(y[test], predict_class, average='binary')
    print('Precision binary: %.3f' % p)
    pbinary.append(p)
    p1 = precision_score(y[test], predict_class, average='micro')
    print('Precision micro: %.3f' % p1)
    pmicro.append(p1)
    p2 = precision_score(y[test], predict_class, average='macro')
    print('Precision macro: %.3f' % p2)
    pmacro.append(p2)
    r = recall_score(y[test], predict_class, average='binary')
    print('Recall binary: %.3f' % r)
    rbi.append(r)
    r1 = recall_score(y[test], predict_class, average='micro')
    print('Recall micro: %.3f' % r1)
    rmicro.append(r1)
    r2 = recall_score(y[test], predict_class, average='macro')
    print('Recall macro: %.3f' % r2)
    rmacro.append(r2)
    s = f1_score(y[test], predict_class, average='binary')
    print('F-Measure: %.3f' % s)
    f.append(s)

    ac = accuracy_score(y[test], predict_class)
    a.append(ac)
    
    
    #Evaluation Measures
    TP = np.diag(conf_mat)
    print("True Positive: ", TP)
    FP = np.sum(conf_mat, axis=0) - TP
    print("False Positive: ", FP)
    FN = np.sum(conf_mat, axis=1) - TP
    print("False Negative", FN)
    num_classes = 2
    TN = []
    for i in range(num_classes):
        temp = np.delete(conf_mat, i, 0)    # delete ith row
        temp = np.delete(temp, i, 1)  # delete ith column
        TN.append(sum(sum(temp)))
    print("True negative: ", TN)

    acc = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    
    print("Accuracy: ", acc)

    print("precision: ", precision)
    print("recall: ", recall)
    F1 = 2 * ((precision * recall)/(precision + recall))
    tp.append(TP)
    tn.append(TN)
    fp.append(FP)
    fn.append(FN)
    
    
    metrics_acc.append(acc)
    metrics_precision.append(precision)
    metrics_recall.append(recall)
    metrics_f1.append(F1)
    print ("F1 measure: ", F1)

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
        label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()



In [ ]:
print("precision macro: ", np.array(pmacro).mean())
print("precision micro: ", np.array(pmicro).mean())
print("precision binary macro: ", np.array(pbinary).mean())
print("accuracy: ", np.array(a).mean())
print("recall macro: ", np.array(rmacro).mean())
print("recall micro: ", np.array(rmicro).mean())
print("recall binary macro: ", np.array(rbi).mean())
print("f1: ", np.array(f).mean())

In [ ]:
#micro-average classwise
micro_precision_group_0 = (tp[0][0] + tp[1][0] + tp[2][0] + tp[3][0] + tp[4][0])/(tp[0][0] + tp[1][0] + tp[2][0] + tp[3][0] + 
                           tp[4][0] + fp[0][0] + fp[1][0] + fp[2][0] + fp[3][0] + fp[4][0])

micro_precision_group_1 = (tp[0][1] + tp[1][1] + tp[2][1] + tp[3][1] + tp[4][1])/(tp[0][1] + tp[1][1] + tp[2][1] + tp[3][1] + 
                           tp[4][1] + fp[0][1] + fp[1][1] + fp[2][1] + fp[3][1] + fp[4][1])

'''micro_precision_group_2 = (tp[0][2] + tp[1][2] + tp[2][2] + tp[3][2] + tp[4][2])/(tp[0][2] + tp[1][2] + tp[2][2] + tp[3][2] + 
                           tp[4][2] + fp[0][2] + fp[1][2] + fp[2][2] + fp[3][2] + fp[4][2])

micro_precision_group_3 = (tp[0][3] + tp[1][3] + tp[2][3] + tp[3][3] + tp[4][3])/(tp[0][3] + tp[1][3] + tp[2][3] + tp[3][3] + 
                           tp[4][3] + fp[0][3] + fp[1][3] + fp[2][3] + fp[3][3] + fp[4][3])
'''
micro_recall_group_0 = (tp[0][0] + tp[1][0] + tp[2][0] + tp[3][0] + tp[4][0])/(tp[0][0] + tp[1][0] 
                        + tp[2][0] + tp[3][0] + tp[4][0] + fn[0][0] + fn[1][0] + fn[2][0] + fn[3][0] + fn[4][0])

micro_recall_group_1 = (tp[0][1] + tp[1][1] + tp[2][1] + tp[3][1] + tp[4][1])/(tp[0][1] + tp[1][1] 
                        + tp[2][1] + tp[3][1] + tp[4][1] + fn[0][1] + fn[1][1] + fn[2][1] + fn[3][1] + fn[4][1])
'''
micro_recall_group_2 = (tp[0][2] + tp[1][2] + tp[2][2] + tp[3][2] + tp[4][2])/(tp[0][2] + tp[1][2] 
                        + tp[2][2] + tp[3][2] + tp[4][2] + fn[0][2] + fn[1][2] + fn[2][2] + fn[3][2] + fn[4][2])

micro_recall_group_3 = (tp[0][3] + tp[1][3] + tp[2][3] + tp[3][3] + tp[4][3])/(tp[0][3] + tp[1][3] 
                        + tp[2][3] + tp[3][3] + tp[4][3] + fn[0][3] + fn[1][3] + fn[2][3] + fn[3][3] + fn[4][3])'''

In [ ]:
micro_precision = []
micro_precision.append(micro_precision_group_0)
micro_precision.append(micro_precision_group_1)
#micro_precision.append(micro_precision_group_2)
#micro_precision.append(micro_precision_group_3)

micro_recall = []
micro_recall.append(micro_recall_group_0)
micro_recall.append(micro_recall_group_1)
#micro_recall.append(micro_recall_group_2)
#micro_recall.append(micro_recall_group_3)



In [ ]:
mirco_f1_group = []

mirco_f1_group.append(2 * ((micro_precision_group_0 * micro_recall_group_0)/(micro_precision_group_0 
                                                                                 + micro_recall_group_0)))
mirco_f1_group.append(2 * ((micro_precision_group_1 * micro_recall_group_1)/(micro_precision_group_1 
                                                                                 + micro_recall_group_1)))
'''mirco_f1_group.append(2 * ((micro_precision_group_2 * micro_recall_group_2)/(micro_precision_group_2 
                                                                                 + micro_recall_group_2)))
mirco_f1_group.append(2 * ((micro_precision_group_3 * micro_recall_group_3)/(micro_precision_group_3 
                                                                                 + micro_recall_group_3)))

#f1_score = 2 * ((micro_precision * micro_recall)/(micro_precision + micro_recall))'''

In [ ]:
print(micro_precision)
print(micro_recall)
print(mirco_f1_group)

In [ ]:
#macro-average classwise
macro_precision_group_0 = (metrics_precision[0][0] + metrics_precision[1][0] + metrics_precision[2][0] + 
                           metrics_precision[3][0] + metrics_precision[4][0])/5
macro_precision_group_1 = (metrics_precision[0][1] + metrics_precision[1][1] + metrics_precision[2][1] + 
                           metrics_precision[3][1] + metrics_precision[4][1])/5
'''macro_precision_group_2 = (metrics_precision[0][2] + metrics_precision[1][2] + metrics_precision[2][2] + 
                           metrics_precision[3][2] + metrics_precision[4][2])/5
macro_precision_group_3 = (metrics_precision[0][3] + metrics_precision[1][3] + metrics_precision[2][3] + 
                           metrics_precision[3][3] + metrics_precision[4][3])/5

macro_recall_group_0 = (metrics_recall[0][0] + metrics_recall[1][0] + metrics_recall[2][0] + 
                           metrics_recall[3][0] + metrics_recall[4][0])/5
macro_recall_group_1 = (metrics_recall[0][1] + metrics_recall[1][1] + metrics_recall[2][1] + 
                           metrics_recall[3][1] + metrics_recall[4][1])/5
macro_recall_group_2 = (metrics_recall[0][2] + metrics_recall[1][2] + metrics_recall[2][2] + 
                           metrics_recall[3][2] + metrics_recall[4][2])/5
macro_recall_group_3 = (metrics_recall[0][3] + metrics_recall[1][3] + metrics_recall[2][3] + 
                           metrics_recall[3][3] + metrics_recall[4][3])/5

macro_f1_group_0 = 2 * ((macro_precision_group_0 * macro_recall_group_0)/(macro_precision_group_0 + macro_recall_group_0))

macro_f1_group_1 = 2 * ((macro_precision_group_1 * macro_recall_group_1)/(macro_precision_group_1 + macro_recall_group_1))

macro_f1_group_2 = 2 * ((macro_precision_group_2 * macro_recall_group_2)/(macro_precision_group_2 + macro_recall_group_2))

macro_f1_group_3 = 2 * ((macro_precision_group_3 * macro_recall_group_3)/(macro_precision_group_3 + macro_recall_group_3))

In [ ]:
macro_precision = []
macro_precision.append(macro_precision_group_0)
macro_precision.append(macro_precision_group_1)
macro_precision.append(macro_precision_group_2)
macro_precision.append(macro_precision_group_3)

macro_recall = []
macro_recall.append(macro_recall_group_0)
macro_recall.append(macro_recall_group_0)
macro_recall.append(macro_recall_group_0)
macro_recall.append(macro_recall_group_0)

marco_f1_group = []

marco_f1_group.append(2 * ((macro_precision_group_0 * macro_recall_group_0)/(macro_precision_group_0 
                                                                                 + macro_recall_group_0)))
marco_f1_group.append(2 * ((macro_precision_group_1 * macro_recall_group_1)/(macro_precision_group_1 
                                                                                 + macro_recall_group_1)))
marco_f1_group.append(2 * ((macro_precision_group_2 * macro_recall_group_2)/(macro_precision_group_2 
                                                                                 + macro_recall_group_2)))
marco_f1_group.append(2 * ((macro_precision_group_3 * macro_recall_group_3)/(macro_precision_group_3 
                                                                                 + macro_recall_group_3)))

#f1_score = 2 * ((micro_precision * micro_recall)/(micro_precision + micro_recall))

print(macro_precision)
print(macro_recall)
print(marco_f1_group)

In [ ]:
# mean accuracy classwise
mean_accuracy = []
for i in range(0, classes):
    a = []
    for j in range(0, 5): #row
        a.append(metrics_acc[j][i])
        n = np.array(a)
    mean_accuracy.append(n.mean())

In [ ]:
mean_accuracy

In [ ]:
metrics_acc